# TME 1 : Prédiction de gènes - Détection de gènes candidats

Le but de ce TME est d’annoter les régions dans le génome qui correspondent à des gènes. Nous allons prendre en compte les différents éléments qui définissent un gène pour pouvoir déterminer les candidats à des régions codantes : phases ouvertes de lecture, propriétés statistiques du code génétique, et comparer les résultats avec l’annotation qui est disponible. 
Nous nous baserons sur le génome de _E. coli_ et ses annotations.


**Nom etudiant 1 :**

**Nom etudiant 2 :**

### A) Mise en route,  télécharger et preparer les données

<b>Question 1)</b> Adapter les commandes du TME précédent pour télécharger le génome de _E. coli_ ainsi que ses annotations


ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_genomic.fna.gz

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_feature_table.txt.gz


In [6]:
import urllib.request
#Utliser la fonction urllib.request.urlretrieve(path)

ImportError: No module named request

In [7]:
#download _genomic.fna.gz


In [8]:
#download _feature_table.txt.gz



<b>Question 2)</b> Decompresser les fichiers

In [9]:
import gzip
import shutil

In [10]:
#Decompresser _genomic.fna.gz


In [11]:
#Decompresser _feature_table.txt.gz


<b>Question 3)</b> Exécuter le code ci-dessous qui va créer un dictionnaire pour représenter le code genetique. Vous avez également deux jeu de séquences pour tester les prochaines fonctions.

In [12]:
# Dictionnaire codons -> acides aminés. Les codons stops sont représentés
# avec le caractère "*"
CODEGENETIQUE = {
    "TTT": "F", "TTC": "F","TTA": "L","TTG": "L","TCT": "S","TCC": "S","TCA": "S","TCG": "S","TAT": "Y","TAC": "Y",
    "TAA": "*","TAG": "*","TGT": "C","TGC": "C","TGA": "*","TGG": "W","CTT": "L","CTC": "L","CTA": "L","CTG": "L",
    "CCT": "P","CCC": "P","CCA": "P","CCG": "P","CAT": "H","CAC": "H","CAA": "Q","CAG": "Q","CGT": "R","CGC": "R",
    "CGA": "R","CGG": "R","ATT": "I","ATC": "I","ATA": "I","ATG": "M","ACT": "T","ACC": "T","ACA": "T","ACG": "T",
    "AAT": "N","AAC": "N","AAA": "K","AAG": "K","AGT": "S","AGC": "S","AGA": "R","AGG": "R","GTT": "V","GTC": "V",
    "GTA": "V","GTG": "V","GCT": "A","GCC": "A","GCA": "A","GCG": "A","GAT": "D","GAC": "D","GAA": "E","GAG": "E",
    "GGT": "G","GGC": "G","GGA": "G","GGG": "G"
}

# Utiliser les sequences TESTSEQ et TESTSEQCLEAN pour tester vos fonctions
TESTSEQ = "ATGAAACGCATTAGCMMCACCATTACCACCACCATCACCATTACCACAGKTAACGGTGCGGGCTGA"
TESTSEQCLEAN = "ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA"


### B) Annotation des phases ouvertes de lecture et traduction en séquence protéique

<b>Question 1)</b> Parfois dans les génomes (représentés par des fichiers numériques), nous trouvons des nucléotides qui n'ont pas été correctement identifiés par la machine de séquençage. Certains instruments nous restreignent les possibilités à un sous-ensemble de nucléotides. Ecrire une fonction `remplace_non_identifies` qui remplace les nucléotides non identifiés par une des possibilités listées ci-dessous de façon aléatoire.<br>

R = G,A (purine)<br>
Y = T,C (pyrimidine)<br>
K = G,T (céto)<br>
M = A,C (amino)<br>
S = G,C (obligations solides)<br>
W = A,T (Les liaisons faibles)<br>
B = G,T,C (tous sauf A)<br>
D = G,A,T (tous sauf C)<br>
H = A,C,T (tous sauf G)<br>
V = G,C,A (tous sauf T)<br>
N = A,G,C,T (any)<br>
X = A,G,C,T (any)<br>



In [13]:
import doctest # C’est pour pouvoir utiliser doctest.testmod() et tester les fonctions
import random

In [14]:
random.choice(['A', 'C', 'T', 'G'])

'G'

In [15]:
##### Question 1

def remplace_non_identifies(seq):
    """
    Remplace les nucléotides non identifiés par une des possibilités de façon aléatoire.
    entrée seq: sequence ayant peut-etre des nucleotides non identifiés
    sortie    : sequence (nettoyé) sans nucléotides non identifiés 
    
    """
    options = {
        "R": ["G", "A"],
        "Y": ["T", "C"],
        "K": ["G", "T"],
        "M": ["A", "C"],
        "S": ["G", "C"],
        "W": ["A", "T"],
        "B": ["G", "T", "C"],
        "D": ["G", "A", "T"],
        "H": ["A", "C", "T"],
        "V": ["G", "C", "A"],
        "N": ["A", "G", "C", "T"],
        "X": ["A", "G", "C", "T"]
    }
   
    chaine=""
    for i in seq:
        if (i in options.keys()): 
            chaine+=random.choice(options[i])
        else: 
            chaine+=i
    return chaine


doctest.testmod()

print(remplace_non_identifies(TESTSEQ))


ATGAAACGCATTAGCCACACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA


<b>Question 2)</b> Ecrire une fonction `listecodon` qui renvoie une liste de codons pour une séquence passée en paramètre. Par exemple, si on passe la séquence ``AACGTGGCA`` comme paramètre votre fonction doit renvoyer ``[‘AAC’, ‘GTG’, ‘GCA’]``. Si la longueur de la séquence n'est pas un multiple de 3 on ne tiendra pas compte des 1 ou 2 nucléotides restant à la fin.


In [16]:
#Pour avoir une aide sur la documentation de python
?range

In [18]:
# Question 2
def listecodon(seq):
    """
    Renvoie une liste de codons pour une séquence passée en paramètre.
    entrée seq : sequence de nucléotides
    sortie     : list de codons de la sequence d'entrée

    Si la longueur de la séquence n'est pas un multiple de 3 elle ne tiendra pas 
    compte des 1 ou 2 nucléotides restant à la fin.
    
    >>> listecodon('AAACCC')
    ['AAA', 'CCC']
    >>> listecodon('AAACC')
    ['AAA']
    >>> listecodon('AAAC')
    ['AAA']
    """
    
    t=len(seq)%3 
    liste=[]
    sttmp=""
    codon=""

    i=0
 
    while(i<(len(seq)-t) ):
        codon+= seq[i]
        codon+= seq[i+1]
        codon+= seq[i+2]
        i=i+3
        liste.append(codon)
        codon=""
    return liste

doctest.testmod()

assert listecodon(TESTSEQCLEAN) == ['ATG', 'AAA', 'CGC', 'ATT', 'AGC', 'ACC', 'ACC', 'ATT', 'ACC', 'ACC','ACC', 'ATC', 'ACC', 'ATT', 'ACC', 'ACA', 'GGT', 'AAC', 'GGT', 'GCG','GGC','TGA']
    
    
print(listecodon(TESTSEQCLEAN))

['ATG', 'AAA', 'CGC', 'ATT', 'AGC', 'ACC', 'ACC', 'ATT', 'ACC', 'ACC', 'ACC', 'ATC', 'ACC', 'ATT', 'ACC', 'ACA', 'GGT', 'AAC', 'GGT', 'GCG', 'GGC', 'TGA']


<b>Question 3)</b> Ecrire une fonction `reversecompl` qui renvoie le brin complémentaire d’une séquence passée en paramètre. Par exemple, si on passe la séquence ``AACGTGGCA`` comme paramètre votre fonction doit renvoyer ``TGCCACGTT``.


In [19]:
# Question 3
def reversecompl(seq):
    """Renvoie le brin complémentaire d’une séquence.
    entrée seq : sequence de nucléotides (brin sens)
    sortie     : sequence de nucléotides (brin complementaire)
    >>> reversecompl('AACGTGGCA')
    'TGCCACGTT'
    """
    compl = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A'}
    chaine=[compl[base] for base in seq ]
    chaine.reverse()
    
    #strr=""
    
    #for i in chaine:
        #strr+= i
    #return strr
  
    return ''.join(chaine)

print (reversecompl('AACGTGGCA'))
print(TESTSEQCLEAN)
assert reversecompl(TESTSEQCLEAN) == "TCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAATGGTGGTGCTAATGCGTTTCAT"

TGCCACGTT
ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA


<b>Question 4-a)</b> Ecrire une fonction `trouve_positions_orfs` qui etant donnée une liste de codon retourne les positions (start, stop) de cadre ouverts de la liste des codons, c'est à dire la séquence commençant par un des codons start: ``ATG``, ``GTG``, ``TTG`` (``ATG`` est le plus fréquent) et finissant par un des codons stop : ``TAA``, ``TAG``, ``TGA``.

Note : si plusieurs start sont trouvés dans la liste de codons, l'orf débute par conventions au start le plus en amont. <br>


In [20]:
def trouver_positions_orfs(codons):
    """Retourne les positions de cadre ouverts de la liste des codons.
    entrée codons: liste de codons
    sortie       : liste de positions [start] et [stop] de cadre ouverts
    
    >>> trouver_positions_orfs(['ATG', 'AAA', 'ATG', 'ATT', 'TAG', 'ATG', 'ACC', 'ATT', 'ACC', 'ACC', 'ACC', 'ATC', 'ACC', 'ATT', 'ACC', 'ACA', 'GGT', 'AAC', 'TGA', 'GGT', 'GCG', 'GGC'])
    ([0, 5], [4, 18])
    """
    starts = [i for i, codon in enumerate(codons) if codon in {'ATG', 'GTG', 'TTG'}] #positions de tous les starts dans codons
    stops =  [i for i, codon in enumerate(codons) if codon in {'TAA', 'TAG', 'TGA'}] #positions de tous les stops  dans codons
    #print( starts)
    #print( stops)
    orf_starts = [] #renvoie la liste de positions de codons start
    orf_stops = []  #renvoie la liste de positions de codons stop, les deux liste ont la meme taille et sont de pair start/stop 


 
    if len(starts)== 0 or len(stops) == 0:
        return (orf_starts, orf_stops)
    
    pos_deb = 0
    pos_fin = 0
    last_starts_index = 0
    last_stops_index = 0
    
    orf_starts.append(starts[pos_deb])

    tmp = 1
    
    while pos_deb < len(starts) and pos_fin < len(stops):
        if tmp%2 == 0: 
            pos_deb = pos_deb + 1
            if pos_deb >= len(starts):
                break
            if(starts[pos_deb] > stops[last_stops_index]):
                orf_starts.append(starts[pos_deb])
                last_starts_index = pos_deb
                tmp = tmp + 1
            
        else: 
            if(stops[pos_fin] > starts[last_starts_index]):
                orf_stops.append(stops[pos_fin])
                last_stops_index = pos_fin
                tmp = tmp + 1
            pos_fin = pos_fin + 1
    
    if tmp%2 == 1:
        orf_starts.pop()

    return (orf_starts, orf_stops)



#trouver_positions_orfs(['AAA', 'TGA', 'TGT', 'AAT', 'AGT', 'GTT', 'TTG', 'ATT', 'AGG', 'GCA'])
#doctest.testmod()

In [21]:
trouver_positions_orfs(['ATG', 'AAA', 'ATG', 'ATT', 'TAG', 'ATG', 'ACC', 'ATT', 'ACC', 'ACC', 'ACC', 'ATC', 'ACC', 'ATT', 'ACC', 'ACA', 'GGT', 'AAC', 'TGA', 'GGT', 'GCG', 'GGC'])
#trouver_positions_orfs(['AAA','TGA','TGT','AAT','AGT','GTT','TTG','ATT','AGG','GCA'])

([0, 5], [4, 18])

<b>Question 4-b)</b> Ecrire une fonction `liste_orfs` qui retourne la liste de tous les cadres ouverts de lectures, c'est à dire les séquence commençant par un des codons start: ``ATG``, ``GTG``, ``TTG`` (``ATG`` est le plus fréquent) et finissant par un des codons stop : ``TAA``, ``TAG``, ``TGA``. Vous renverrez les séquences pour les 6 phases de lecture (3 pour le brin sens et 3 pour le brin complémentaire). Ces séquences sont nommées séquences CDS (pour _CoDing Sequences_ en anglais) et les phases ouvertes de lecture ORFS (pour _Open Reading Frame_). <br>

**Note 1**: Coder d'abord la fonction `liste_orfs_sens` qui retourne la liste de trois cadres ouverts de lectures du brin sens. Pour retourver la liste de trois cadres ouverts de lectures du brin complementaire, vous pouvez combiner les deux fonctions `reversecompl` et `liste_orfs_sens`

**Note 2**: Vous pouvez utiliser les fonctions precedents tels que `listecodon` et `trouver_position_orfs`




In [22]:
# Question 4

def liste_orfs_sens(seq):
    """
    Liste tous les cadres ouverts de lectures du brin sens.
    entrée : sequence de nucléotides 
    sortie : liste contenant tous les cadres ouverts de lectures
    >>> sorted(liste_orfs_sens('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
    ['ATGATGTAA', 'GTGTTTTGA']
    """
    
    
    orfs = []
    cadres = [listecodon(seq), listecodon(seq[1:]), listecodon(seq[2:])]
    for i in range(3):
        (start_liste, stop_liste) = trouver_positions_orfs(cadres[i])
        for index in range(len(start_liste)):
            pos_deb = start_liste[index]
            pos_fin = stop_liste[index]
            orfs.append(''.join(cadres[i][pos_deb:pos_fin+1]))
    return orfs


    sorted(liste_orfs_sens('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))


#doctest.testmod()

In [23]:
TestSequence = "AAAATGCGCATTAGCACCACCTGAATTACATGACCATCACCATTACCACATGAAACGGTGCGGGC"
liste_orfs_sens(TestSequence)
sorted(liste_orfs_sens('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))

['ATGATGTAA', 'GTGTTTTGA']

Comparer vos resultats avec ceux de la fonction ci-dessus qui utilise Regular Expression

In [24]:
import re


In [25]:
[
    match.group()
    for match in re.finditer('((ATG|GTG|TTG)((?!TAA|TAG|TGA)([ACTG][ACTG][ACTG]))*(TAA|TAG|TGA))',
                             'AAATGATGTAATAGTGTTTTGATTAGGGCAT')
]

['ATGATGTAA', 'GTGTTTTGA']

In [26]:
def __liste_orfs_sens(seq):
    """Retourne la liste des cadres ouverts de lecture, sens 5' vers 3'.
    entrée : sequence de nucléotides 
    sortie : liste contenant tous les cadres ouverts de lectures
    """
    return [
        match.group() for match in re.finditer(
            '((ATG|GTG|TTG)((?!TAA|TAG|TGA)([ACTG][ACTG][ACTG]))*(TAA|TAG|TGA))',
            seq)
    ]


def __liste_orfs(seq):
    """
    Retourne la liste de tous les cadres ouverts de lectures.
    entrée : sequence de nucléotides
    sortie : liste des cadres ouverts de lecture
        
    >>> sorted(liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
    ['ATGATGTAA', 'GTGTTTTGA']
    """
    liste = __liste_orfs_sens(seq)
    liste.extend(__liste_orfs_sens(reversecompl(seq)))
    return liste

sorted(__liste_orfs_sens('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
#doctest.testmod()
#['ATGATGTAA', 'GTGTTTTGA']

print(__liste_orfs('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))
print(__liste_orfs_sens('AAATGATGTAATAGTGTTTTGATTAGGGCAT'))

['ATGATGTAA', 'GTGTTTTGA', 'ATGCCCTAA']
['ATGATGTAA', 'GTGTTTTGA']


In [27]:
__liste_orfs(TestSequence)

['ATGCGCATTAGCACCACCTGA',
 'ATGACCATCACCATTACCACATGA',
 'ATGTGGTAA',
 'GTGATGGTCATGTAA']